# QAtools Notebook

## Project and Flight Information

In [2]:
project = 'SPICULE'
flight = 'rf12'

Quick-look plots and statistics from field-phase NCAR/RAF flight netCDF datasets.

Assign variables `project` and `flight` and 'Run All Cells'.

References:

README: 

NCAR/RAF netCDF Format: https://archive.eol.ucar.edu/raf/software/netCDF.html

In [3]:
import netCDF4
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import itertools 
from bokeh.io import push_notebook, show, output_notebook
from bokeh.layouts import row
from bokeh.plotting import figure, show
from bokeh.models import Title, CustomJS, Select, TextInput, Button, LinearAxis, Range1d
from bokeh.palettes import Category10
from metpy.plots import SkewT
from metpy.units import pandas_dataframe_to_unit_arrays, units
from datetime import datetime
from IPython.display import display
output_notebook()

ModuleNotFoundError: No module named 'netCDF4'

In [ ]:
# function definition for creating timeseries plot
def timeseries_plot(title, y_label, dataframe):
    colors = itertools.cycle(Category10[8])
    plot = figure(width=1200, height=400, title=title) 
    plot.add_layout(Title(text="Time [Seconds since Midnight]", align="center"), "below")
    plot.add_layout(Title(text=y_label, align="center"), "left")
    # loop over variables in the pandas dataframe to add to plot
    for i in asc[dataframe].columns:
        try:
            plot.line(asc['Time'], asc[dataframe][i], color=next(colors), legend_label=i)
        except Exception as e:
            print(e)
        try:
            plot.legend.location = "top_left"
        except Exception as e:
            print(e)
    try:
        show(plot)
    except Exception as e:
        print(e)

In [ ]:
filepath = '/scr/raf_data/'
input_file = project + flight + '.nc'
nc = netCDF4.Dataset(filepath + '/' + project + '/' + input_file, mode='r')

asc = {}
units = {}
# loop over keys in netCDF file and organize
for i in nc.variables.keys():
    dims = str(nc.variables[i].dimensions)
    if dims == "('Time',)":
        output = nc[i][:]
        asc[i] = pd.DataFrame(output)
        units_var = nc.variables[i].getncattr('units')
        units[i] = pd.Series(units_var)
        asc[i].columns = pd.MultiIndex.from_tuples(zip(asc[i].columns, units[i]))
    # accommodate histogram data
    elif "sps1" in dims:
        histo_output = pd.DataFrame(nc.variables[i][:, 0, :])
        asc[i] = pd.DataFrame(histo_output)
        try:
            cellsize = nc.variables[i].getncattr('CellSizes')
            cellsize[i] = pd.Series(data=cellsize)
            asc[i].columns = pd.MultiIndex.from_tuples(zip(asc[i].columns, cellsize[i]))

        except:
            asc[i].columns = pd.MultiIndex.from_tuples(zip(asc[i].columns, asc[i].columns))
    else:
        pass

# concatenate the pandas dataframe
asc = pd.concat(asc, axis=1, ignore_index=False)
dtime = nc.variables['Time']
dtime = netCDF4.num2date(dtime[:], dtime.units)
asc.columns = asc.columns.droplevel(1)
asc.columns = asc.columns.droplevel(1)
colors = itertools.cycle(Category10[6])

In [ ]:
# filter based on ground speed
asc.drop(asc[asc['GGSPD'] < 20].index, inplace = True)

## Flight Track

In [ ]:
flight_track = []
flight_track.append("GGLAT")
flight_track.append("GGLON")
plot = figure(width=800, height=800, title=project + ' ' + flight + " Flight Track") 
plot.add_layout(Title(text="Longitude [Degrees]", align="center"), "below")
plot.add_layout(Title(text="Latitude [Degrees]", align="center"), "left")
try:
    plot.line(asc[flight_track]["GGLON"], asc[flight_track]["GGLAT"], color="blue")
    show(plot)
except Exception as e:
    print(e)

## Altitude, Heading, and GPS Quality

In [ ]:
alt_heading = []
for i in asc.columns:
    if i.startswith('GGALTF') | i.startswith('PALTF') | i.startswith('PA2'):
        alt_heading.append(i)
asc[alt_heading].describe()

In [ ]:
timeseries_plot("Altitude", "Altitude [Feet]", alt_heading)

colors = itertools.cycle(Category10[6])
plot = figure(width=1200, height=400, title="Heading") 
plot.add_layout(Title(text="Time [Seconds since Midnight]", align="center"), "below")
plot.add_layout(Title(text="True Heading [Degrees]", align="center"), "left")
try:
    plot.line(asc['Time'], asc['THDG'], color=next(colors))
    show(plot)
except Exception as e:
    print(e)

colors = itertools.cycle(Category10[6])
plot = figure(width=1200, height=400, title="GPS Quality: 5 = TerraStar corrected, 2 = TerraStar converging, 9 = WAAS, 1 = Standard GPS, 0 = No Fix") 
plot.add_layout(Title(text="Time [Seconds since Midnight]", align="center"), "below")
plot.add_layout(Title(text="GPS Quality []", align="center"), "left")
try:
    plot.line(asc['Time'], asc['GGQUAL'], color=next(colors), legend_label="GGQUAL")
except Exception as e:
    print(e)
try:
    plot.line(asc['Time'], asc['GGNSAT'], color=next(colors), legend_label="GGNSAT")
    show(plot)
except Exception as e:
    print(e)

## Temperatures

### Air Temperatures

In [ ]:
temp = []
for i in asc.columns:
    if i.startswith('ATX') | i.startswith('ATH') | i.startswith('ATF') | i.startswith('AT_'):
        temp.append(i)
try:
    temp.remove('AT_VXL')
except Exception as e:
    print(e)
asc[temp].describe()

In [ ]:
timeseries_plot("Temperature", "Temperature [\N{DEGREE CELSIUS}]", temp)
    
colors = itertools.cycle(Category10[6])
plot = figure(width=1200, height=400, title="ATX - Temperature") 
plot.add_layout(Title(text="Time [Seconds since Midnight]", align="center"), "below")
plot.add_layout(Title(text="Temperature Difference [\N{DEGREE CELSIUS}]", align="center"), "left")
for i in asc[temp].columns:
    try:
        plot.line(asc['Time'], asc['ATX'] - asc[temp][i], color=next(colors), legend_label=i)
    except Exception as e:
        print(e)
try:
    plot.legend.location = "top_left"
    show(plot)
except Exception as e:
    print(e)
    
# loop over temperatures
for i in asc[temp].columns:    
    try:
        # create plot labels
        plot = figure(width=600, height=400, title=asc[temp][i].name) 
        plot.add_layout(Title(text="ATH1 [\N{DEGREE CELSIUS}]", align="center"), "below")
        plot.add_layout(Title(text=asc[temp][i].name + " [\N{DEGREE CELSIUS}]", align="center"), "left")
    except Exception as e:
        print(e)
    try:
        # create scatter plots
        plot.scatter(asc[temp]['ATX'], asc[temp]['ATX'], color = 'black', line_dash = 'dotted', legend_label='1:1')
        plot.scatter(asc[temp]['ATH1'], asc[temp][i], color='orange', legend_label=i)
        
        # create additional y axis range
        plot.extra_y_ranges['delta'] = Range1d(-4, 4)
        ax2 = LinearAxis(y_range_name="delta", axis_label="Delta Temperature [C]")
        ax2.axis_label_text_color ="red"
        plot.add_layout(ax2, 'right')
        
        # create line plot of temperature differences using the additional y axis range
        plot.line(asc[temp]['ATH1'], asc[temp][i]-asc[temp]['ATH1'], color='red', legend_label=i, y_range_name="delta")
        plot.line(asc[temp]['ATH1'], 1, color = 'gray', y_range_name="delta")
        plot.line(asc[temp]['ATH1'], -1, color = 'gray', y_range_name="delta")
    except Exception as e:
        print(e)
    try:
        plot.legend.location = "top_left"
    except Exception as e:
        print(e)
    show(plot)

### Dew Point Temperatures

In [ ]:
dpt = []
for i in asc.columns:
    if i.startswith('DP_') | i.startswith('ATX'):
        dpt.append(i)
asc[dpt].describe()

In [ ]:
timeseries_plot("Dew Point Temperatures", "Dew Point [C]", dpt)

colors = itertools.cycle(Category10[6])
plot = figure(width=1200, height=400, title="Diff Dew Point Temperatures") 
plot.add_layout(Title(text="Time [Seconds since Midnight]", align="center"), "below")
plot.add_layout(Title(text="Diff DPL - DP Dew Point [C]", align="center"), "left")
for i in asc[dpt].columns:
    try:
        plot.line(asc['Time'], asc['DP_DPL'] - asc[dpt][i], color=next(colors), legend_label=i)
    except Exception as e:
        print(e)
try:
    plot.legend.location = "top_left"
    show(plot)
except Exception as e:
    print(e)   

## Pressure

In [ ]:
# create plot labels
plot = figure(width=900, height=900) 
plot.add_layout(Title(text="DP_DPR [C]", align="center"), "below")
plot.add_layout(Title(text="DP_DPL [C]", align="center"), "left")
try:
    # create scatter plots
    plot.scatter(asc['DP_DPR'], asc['DP_DPL'], color = 'blue', legend_label='DP_DPL')
    plot.line(asc['DP_DPR'], asc['DP_DPR'], color = 'black', legend_label='1:1')

    plot.line(asc['DP_DPR'], asc['DP_DPR']+5, color = 'orange', legend_label='+-5C')
    plot.line(asc['DP_DPR'], asc['DP_DPR']-5, color = 'orange')

except Exception as e:
    print(e)
try:
    plot.legend.location = "top_left"
except Exception as e:
    print(e)
show(plot)

### Cavity Pressure

In [ ]:
cav_pressure = []
for i in asc.columns:
    if i.startswith('CAVP_') | i.startswith('CAVPF_') | i.startswith('PSXC'):
        cav_pressure.append(i)
asc[cav_pressure].describe()

In [ ]:
colors = itertools.cycle(Category10[8])
plot = figure(width=1200, height=400, title="Cavity Pressure") 
plot.add_layout(Title(text="Time [Seconds since Midnight]", align="center"), "below")
plot.add_layout(Title(text="Pressure [hPa]]", align="center"), "left")
for i in asc[cav_pressure].columns:
    try:
        plot.line(asc['Time'], asc[cav_pressure][i], color=next(colors), legend_label=i)
    except Exception as e:
        print(e)
try:
    plot.legend.location = "top_left"
    show(plot)
except Exception as e:
    print(e)

In [ ]:
vapor_pressure = []
for i in asc.columns:
    if i.startswith('EW_'):
        vapor_pressure.append(i)
try:
    asc[vapor_pressure].describe()
except Exception as e:
    print(e)
    
mixing_ratio = []
for i in asc.columns:
    if i.startswith('MR_'):
        mixing_ratio.append(i)
try:
    asc[mixing_ratio].describe()
except Exception as e:
    print(e)
    
rh = []
for i in asc.columns:
    if i.startswith('RH_'):
        rh.append(i)
try:
    asc[rh].describe()
except Exception as e:
    print(e)

In [ ]:
colors = itertools.cycle(Category10[8])
plot = figure(width=1200, height=400) 
plot.add_layout(Title(text="Time [Seconds since Midnight]", align="center"), "below")
plot.add_layout(Title(text="EWy [hPa]]", align="center"), "left")
for i in asc[vapor_pressure].columns:
    try:
        plot.line(asc['Time'], asc[vapor_pressure][i], color=next(colors), legend_label=i)
    except Exception as e:
        print(e)
try:
    plot.legend.location = "top_left"
    show(plot)
except Exception as e:
    print(e)

### Static Pressure

In [ ]:
static_pressure = []
for i in asc.columns:
    if i.startswith('PS_') | i.startswith('PSF') | i.startswith('PST'):
        static_pressure.append(i)
asc[static_pressure].describe()

In [ ]:
colors = itertools.cycle(Category10[8])
plot = figure(width=1200, height=400, title="Static Pressure") 
plot.add_layout(Title(text="Time [Seconds since Midnight]", align="center"), "below")
plot.add_layout(Title(text="Pressure [hPa]]", align="center"), "left")
for i in asc[static_pressure].columns:
    try:
        plot.line(asc['Time'], asc[static_pressure][i], color=next(colors), legend_label=i)
    except Exception as e:
        print(e)
try:
    plot.legend.location = "top_left"
    show(plot)
except Exception as e:
    print(e)

colors = itertools.cycle(Category10[8])
plot = figure(width=1200, height=400, title="Difference versus PSFC") 
plot.add_layout(Title(text="Time [Seconds since Midnight]", align="center"), "below")
plot.add_layout(Title(text="Pressure - PSFC [C]", align="center"), "left")
for i in asc[static_pressure].columns:
    try:
        plot.line(asc['Time'], asc['PSFC'] - asc[static_pressure][i], color=next(colors), legend_label=i)
    except Exception as e:
        print(e)
try:
    plot.legend.location = "top_left"
    show(plot)
except Exception as e:
    print(e)
    
colors = itertools.cycle(Category10[8])
plot = figure(width=1200, height=400, title="Difference versus PSXC") 
plot.add_layout(Title(text="Time [Seconds since Midnight]", align="center"), "below")
plot.add_layout(Title(text="Pressure - PSXC [C]", align="center"), "left")
for i in asc[static_pressure].columns:
    try:
        plot.line(asc['Time'], asc['PSXC'] - asc[static_pressure][i], color=next(colors), legend_label=i)
    except Exception as e:
        print(e)
try:
    plot.legend.location = "top_left"
    show(plot)
except Exception as e:
    print(e) 

### Dynamic Pressure

In [ ]:
dynamic_pressure_corr = []
for i in asc.columns:
    if i.startswith('QC_') | i.startswith('QCTFC') | i.startswith('QCRC') | i.startswith('QCFC'):
        dynamic_pressure_corr.append(i)
asc[dynamic_pressure_corr].describe()

In [ ]:
dynamic_pressure_uncorr = []
for i in asc.columns:
    if i.startswith('QC_') | i.startswith('QCTF') | i.startswith('QCR') | i.startswith('QCF'):
        dynamic_pressure_uncorr.append(i)
try:
    dynamic_pressure_uncorr.remove('QCRTEMP')
except:
    pass
try:
    dynamic_pressure_uncorr.remove('QCFTEMP')
except:
    pass
try:
    dynamic_pressure_uncorr.remove('QCTFTEMP')
except:
    pass
try:
    dynamic_pressure_uncorr.remove('QCTF')
except:
    pass
asc[dynamic_pressure_uncorr].describe()

In [ ]:
colors = itertools.cycle(Category10[8])
plot = figure(width=1200, height=400, title="Dynamic Pressure") 
plot.add_layout(Title(text="Time [Seconds since Midnight]", align="center"), "below")
plot.add_layout(Title(text="Corrected Pressure QCyC [hPa]]", align="center"), "left")
for i in asc[dynamic_pressure_corr].columns:
    try:
        plot.line(asc['Time'], asc[dynamic_pressure_corr][i], color=next(colors), legend_label=i)
    except Exception as e:
        print(e)
try:
    plot.legend.location = "top_left"
    show(plot)
except Exception as e:
    print(e)

colors = itertools.cycle(Category10[8])
plot = figure(width=1200, height=400, title="Difference versus QCFC") 
plot.add_layout(Title(text="Time [Seconds since Midnight]", align="center"), "below")
plot.add_layout(Title(text="Pressure - QCFC [C]", align="center"), "left")
for i in asc[dynamic_pressure_corr].columns:
    try:
        plot.line(asc['Time'], asc[dynamic_pressure_corr]['QCFC'] - asc[dynamic_pressure_corr][i], color=next(colors), legend_label=i)
    except Exception as e:
        print(e)
try:
    plot.legend.location = "top_left"
    show(plot)
except Exception as e:
    print(e)
    
colors = itertools.cycle(Category10[8])
plot = figure(width=1200, height=400, title="Difference versus PSXC") 
plot.add_layout(Title(text="Time [Seconds since Midnight]", align="center"), "below")
plot.add_layout(Title(text="Pressure - QCXC [C]", align="center"), "left")
for i in asc[dynamic_pressure_uncorr].columns:
    try:
        plot.line(asc['Time'], asc['QCXC'] - asc[dynamic_pressure_uncorr][i], color=next(colors), legend_label=i)
    except Exception as e:
        print(e)
try:
    plot.legend.location = "top_left"
    show(plot)
except Exception as e:
    print(e) 

## Airspeeds

In [ ]:
airspeed = []
for i in asc.columns:
    if i.startswith('TAS'):
        airspeed.append(i)
try:
    airspeed.remove('TASFLG')
except:
    pass
asc[airspeed].describe()

In [ ]:
colors = itertools.cycle(Category10[8])
plot = figure(width=1200, height=400, title="Airspeed") 
plot.add_layout(Title(text="Time [Seconds since Midnight]", align="center"), "below")
plot.add_layout(Title(text="TASy [m/s]]", align="center"), "left")
for i in asc[airspeed].columns:
    try:
        plot.line(asc['Time'], asc[airspeed][i], color=next(colors), legend_label=i)
    except Exception as e:
        print(e)
try:
    plot.legend.location = "top_left"
    show(plot)
except Exception as e:
    print(e)

colors = itertools.cycle(Category10[8])
plot = figure(width=1200, height=400, title="Delta Airspeed") 
plot.add_layout(Title(text="Time [Seconds since Midnight]", align="center"), "below")
plot.add_layout(Title(text="Difference v TASF [m/s]", align="center"), "left")
for i in asc[airspeed].columns:
    try:
        plot.line(asc['Time'], asc[airspeed][i]-asc['TASF'], color=next(colors), legend_label=i)
    except Exception as e:
        print(e)
try:
    plot.legend.location = "top_left"
    show(plot)
except Exception as e:
    print(e)

In [ ]:
mach = []
for i in asc.columns:
    if i.startswith('MACH'):
        mach.append(i)
asc[mach].describe()

In [ ]:
colors = itertools.cycle(Category10[8])
plot = figure(width=1200, height=400, title="Mach") 
plot.add_layout(Title(text="Time [Seconds since Midnight]", align="center"), "below")
plot.add_layout(Title(text="MACHy", align="center"), "left")
for i in asc[mach].columns:
    try:
        plot.line(asc['Time'], asc[mach][i], color=next(colors), legend_label=i)
    except Exception as e:
        print(e)
try:
    plot.legend.location = "top_left"
    show(plot)
except Exception as e:
    print(e)

In [ ]:
pst = []
for i in asc.columns:
    if i.startswith('PST'):
        pst.append(i)
asc[pst].describe()

In [ ]:
timeseries_plot("Corrected Static Pressure Top of Fuselage", "PSTF(C) [hPa]", pst)

## Winds

### Wind Direction

In [ ]:
wd_winds = []
for i in asc.columns:
    if i.startswith('WD') | i.startswith('IWD'):
        wd_winds.append(i)
asc[wd_winds].describe()

In [ ]:
timeseries_plot("Wind Direction", "Wind Speed [m/s]", wd_winds)

### Wind Speeds

In [ ]:
ws_winds = []
for i in asc.columns:
    if i.startswith('WS') | i.startswith('IWS'):
        ws_winds.append(i)
asc[ws_winds].describe()

In [ ]:
timeseries_plot("Wind Speed", "Wind Speed [m/s]", ws_winds)

In [ ]:
wi_winds = []
for i in asc.columns:
    if i.startswith('WIX') | i.startswith('WIC') | i.startswith('WIY'):
        wi_winds.append(i)
asc[wi_winds].describe()

In [ ]:
timeseries_plot("Vertical Wind Speed", "Vertical Wind Speed [m/s]", wi_winds)

In [ ]:
east_wind = []
for i in asc.columns:
    if i.startswith('UIC') | i.startswith('IUX'):
        east_wind.append(i)
asc[east_wind].describe()

In [ ]:
north_wind = []
for i in asc.columns:
    if i.startswith('VIC') | i.startswith('IVY'):
        north_wind.append(i)
asc[north_wind].describe()

In [ ]:
timeseries_plot("East Wind Speed", "East Wind Speed [m/s]", east_wind)
timeseries_plot("North Wind Speed", "North Wind Speed [m/s]", north_wind)

## Schuler Oscillation

In [4]:
colors = itertools.cycle(Category10[8])
plot = figure(width=1200, height=400, title="IRU Schuler oscillation, east west component ") 
plot.add_layout(Title(text="Time [Seconds since Midnight]", align="center"), "below")
plot.add_layout(Title(text="GGVEW [m/s]", align="center"), "left")
try:
    plot.line(asc['Time'], asc["GGVEW"], color=next(colors), legend_label="GGVEW")
except Exception as e:
    print(e)
try:
     plot.line(asc['Time'], asc["VEW"], color=next(colors), legend_label="VEW")
except Exception as e:
    print(e)   

try:
     plot.line(asc['Time'], (asc["GGVEW"] - asc["VEW"]) *50, color=next(colors), legend_label="Difference * 50")
except Exception as e:
    print(e)
    
try:
    plot.legend.location = "top_left"
    show(plot)
except Exception as e:
    print(e)

NameError: name 'itertools' is not defined

In [ ]:
colors = itertools.cycle(Category10[8])
plot = figure(width=1200, height=400, title="IRU Schuler oscillation, north south component ") 
plot.add_layout(Title(text="Time [Seconds since Midnight]", align="center"), "below")
plot.add_layout(Title(text="GGVNS [m/s]", align="center"), "left")
try:
    plot.line(asc['Time'], asc["GGVNS"], color=next(colors), legend_label="GGVEW")
except Exception as e:
    print(e)
try:
     plot.line(asc['Time'], asc["VNS"], color=next(colors), legend_label="VEW")
except Exception as e:
    print(e)   

try:
     plot.line(asc['Time'], (asc["GGVNS"] - asc["VNS"]) *50, color=next(colors), legend_label="Difference * 50")
except Exception as e:
    print(e)
    
try:
    plot.legend.location = "top_left"
    show(plot)
except Exception as e:
    print(e)

## Radome Pressures and Angles

In [ ]:
radome_angles = []
for i in asc.columns:
    if i.startswith('AKRD') | i.startswith('PITCH') | i.startswith('AOAREF'):
        radome_angles.append(i)
asc[radome_angles].describe()

In [ ]:
timeseries_plot("Radome Angles", "Angle [Degrees]", radome_angles)

In [ ]:
colors = itertools.cycle(Category10[8])
plot = figure(width=1200, height=400, title="") 
plot.add_layout(Title(text="Time [Seconds since Midnight]", align="center"), "below")
plot.add_layout(Title(text="SSRD [Degrees]", align="center"), "left")
try:
    plot.line(asc['Time'], asc["SSRD"], color=next(colors), legend_label="SSRD")
except Exception as e:
    print(e)
try:
    plot.legend.location = "top_left"
    show(plot)
except Exception as e:
    print(e)

In [ ]:
colors = itertools.cycle(Category10[8])
plot = figure(width=1200, height=400, title="") 
plot.add_layout(Title(text="Time [Seconds since Midnight]", align="center"), "below")
plot.add_layout(Title(text="ADIFR [hPa]", align="center"), "left")
try:
    plot.line(asc['Time'], asc["ADIFR"], color=next(colors), legend_label="ADIFR")
except Exception as e:
    print(e)
try:
    plot.legend.location = "top_left"
    show(plot)
except Exception as e:
    print(e)

In [ ]:
colors = itertools.cycle(Category10[8])
plot = figure(width=1200, height=400, title="") 
plot.add_layout(Title(text="Time [Seconds since Midnight]", align="center"), "below")
plot.add_layout(Title(text="BDIFR [hPa]", align="center"), "left")
try:
    plot.line(asc['Time'], asc["BDIFR"], color=next(colors), legend_label="BDIFR")
except Exception as e:
    print(e)
try:
    plot.legend.location = "top_left"
    show(plot)
except Exception as e:
    print(e)

In [ ]:
acins = []
for i in asc.columns:
    if i.startswith('ACINS'):
        acins.append(i)
asc[acins].describe()

In [ ]:
timeseries_plot("IRS Vertical Acceleration", "ACINS [m/s^2]", acins)

## Pitch, Roll, and Heading

### Pitch

In [ ]:
pitch = []
for i in asc.columns:
    if i.startswith('PITCH'):
        pitch.append(i)
asc[pitch].describe()

In [ ]:
colors = itertools.cycle(Category10[8])
plot = figure(width=1200, height=400, title="Pitch") 
plot.add_layout(Title(text="Time [Seconds since Midnight]", align="center"), "below")
plot.add_layout(Title(text="PITCH [Degrees]", align="center"), "left")
for i in asc[pitch].columns:
    try:
        plot.line(asc['Time'], asc[pitch][i], color=next(colors), legend_label=i)
    except Exception as e:
        print(e)
    try:
        plot.line(asc['Time'], (asc[pitch]['PITCH'] - asc[pitch]['PITCH_IRS2']) * 50, color=next(colors), legend_label="PITCH - PITCH_IRS2 Difference * 50")
    except Exception as e:
        pass
    try:
        plot.line(asc['Time'], (asc[pitch]['PITCH'] - asc[pitch]['PITCH_IRS3']) * 50, color=next(colors), legend_label="PITCH - PITCH_IRS3 Difference * 50")
    except Exception as e:
        pass
try:
    plot.legend.location = "top_left"
except Exception as e:
    print(e)
    
show(plot)

### Roll

In [ ]:
roll = []
for i in asc.columns:
    if i.startswith('ROLL'):
        roll.append(i)
asc[roll].describe()

In [ ]:
colors = itertools.cycle(Category10[8])
plot = figure(width=1200, height=400, title="Roll") 
plot.add_layout(Title(text="Time [Seconds since Midnight]", align="center"), "below")
plot.add_layout(Title(text="PITCH [Degrees]", align="center"), "left")
for i in asc[roll].columns:
    try:
        plot.line(asc['Time'], asc[roll][i], color=next(colors), legend_label=i)
    except Exception as e:
        print(e)
    try:
        plot.line(asc['Time'], (asc[roll]['ROLL'] - asc[roll]['ROLL_IRS2']) * 500, color=next(colors), legend_label="ROLL - ROLL_IRS2 Difference * 500")
    except Exception as e:
        pass
    try:
        plot.line(asc['Time'], (asc[roll]['ROLL'] - asc[roll]['ROLL_IRS3']) * 500, color=next(colors), legend_label="ROLL - ROLL_IRS3 Difference * 500")
    except Exception as e:
        pass
try:
    plot.legend.location = "top_left"
except Exception as e:
    print(e) 
show(plot)

### Heading

In [ ]:
thdg = []
for i in asc.columns:
    if i.startswith('THDG'):
        thdg.append(i)
asc[thdg].describe()

In [ ]:
colors = itertools.cycle(Category10[8])
plot = figure(width=1200, height=400, title="True Heading") 
plot.add_layout(Title(text="Time [Seconds since Midnight]", align="center"), "below")
plot.add_layout(Title(text="THDG [degrees]", align="center"), "left")

for i in asc[thdg].columns:
    try:
        plot.line(asc['Time'], asc[thdg][i], color=next(colors), legend_label=i)
    except Exception as e:
        print(e)
try:
    plot.legend.location = "top_left"
except Exception as e:
    print(e)  
show(plot)

## ACINS, VSPD, Altitude

### ACINS

In [ ]:
acins = []
for i in asc.columns:
    if i.startswith('ACINS'):
        acins.append(i)
asc[acins].describe()

In [ ]:
colors = itertools.cycle(Category10[8])
plot = figure(width=1200, height=400, title="ACINS") 
plot.add_layout(Title(text="Time [Seconds since Midnight]", align="center"), "below")
plot.add_layout(Title(text="ACINS [m/s^2]", align="center"), "left")

for i in asc[acins].columns:
    try:
        plot.line(asc['Time'], asc[acins][i], color=next(colors), legend_label=i)
    except Exception as e:
        print(e)
try:
    plot.legend.location = "top_left"
except Exception as e:
    print(e)  
show(plot)

### VSPD 

In [ ]:
vspd = []
for i in asc.columns:
    if i.startswith('VSPD'):
        vspd.append(i)
asc[vspd].describe()

In [ ]:
colors = itertools.cycle(Category10[8])
plot = figure(width=1200, height=400, title="VSPD") 
plot.add_layout(Title(text="Time [Seconds since Midnight]", align="center"), "below")
plot.add_layout(Title(text="VSPD", align="center"), "left")

for i in asc[vspd].columns:
    try:
        plot.line(asc['Time'], asc[vspd][i], color=next(colors), legend_label=i)
    except Exception as e:
        print(e)
try:
    plot.legend.location = "top_left"
except Exception as e:
    print(e)  
show(plot)

### Altitude

In [ ]:
alt = []
for i in asc.columns:
    if i.startswith('GGALT') | i.startswith('ALT_'):
        alt.append(i)
try:
    alt.remove('GGALTSD_232')
except:
    pass
try:
    alt.remove('GGALTSD')
except:
    pass

asc[alt].describe()

In [ ]:
colors = itertools.cycle(Category10[8])
plot = figure(width=1200, height=400, title="Altitude") 
plot.add_layout(Title(text="Time [Seconds since Midnight]", align="center"), "below")
plot.add_layout(Title(text="Altitude (ALT) [m]", align="center"), "left")

for i in asc[alt].columns:
    try:
        plot.line(asc['Time'], asc[alt][i], color=next(colors), legend_label=i)
    except Exception as e:
        print(e)
try:
    plot.legend.location = "top_left"
except Exception as e:
    print(e)  
show(plot)

## Skew-T Plot

In [ ]:
from metpy.units import pandas_dataframe_to_unit_arrays, units
plt.rcParams['figure.figsize'] = (20, 20)

p = asc['PSXC']
T = asc['ATX']
Td = asc['DP_DPL']
skew = SkewT()

# Plot the data using normal plotting functions, in this case using
# log scaling in Y, as dictated by the typical meteorological plot
skew.plot(p, T, 'r')
skew.plot(p, Td, 'g')

# Set some better labels than the default
skew.ax.set_xlabel('Temperature [\N{DEGREE CELSIUS}]')
skew.ax.set_ylabel('Pressure [hPa]')

# Add the relevant special lines
skew.plot_dry_adiabats()
skew.plot_moist_adiabats()
skew.plot_mixing_lines()
skew.ax.set_ylim(1000, 100)
skew.ax.set_xlim(-60, 30)

## Potential Temperature

In [ ]:
theta = []
for i in asc.columns:
    if i.startswith('THETA'):
        theta.append(i)

asc[theta].describe()

In [ ]:
timeseries_plot("Potential Temperature", "THETA [K]", theta)

In [ ]:
colors = itertools.cycle(Category10[8])
plot = figure(width=800, height=800, title="") 
plot.add_layout(Title(text="Potential Temperature [K]", align="center"), "below")
plot.add_layout(Title(text="Pressure [hPa]", align="center"), "left")

for i in asc[theta].columns:
    try:
        plot.line(asc[theta][i], asc['PSXC'], color=next(colors), legend_label=i)
    except Exception as e:
        print(e)
try:
    plot.legend.location = "top_left"
except Exception as e:
    print(e)  
show(plot)

## Chemistry

In [ ]:
colors = itertools.cycle(Category10[8])
plot = figure(width=1200, height=400, title="Carbon Dioxide") 
plot.add_layout(Title(text="ppmv", align="center"), "left")

try:
    plot.line(asc['Time'], asc['CO2C_PIC2401'], color=next(colors), legend_label='CO2C_PIC2401')
except Exception as e:
        print(e)
try:
    plot.line(asc['Time'], asc['CO2_PIC2401'], color=next(colors), legend_label='CO2_PIC2401')
except Exception as e:
        print(e)
try:
    plot.legend.location = "top_left"
except Exception as e:
    print(e)  
show(plot)
colors = itertools.cycle(Category10[8])
plot = figure(width=1200, height=400, title="Methane") 
plot.add_layout(Title(text="ppmv", align="center"), "left")

try:
    plot.line(asc['Time'], asc['CH4C_PIC2401'], color=next(colors), legend_label='CH4C_PIC2401')
except Exception as e:
        print(e)
try:
    plot.line(asc['Time'], asc['CH4_PIC2401'], color=next(colors), legend_label='CH4_PIC2401')
except Exception as e:
        print(e)
try:
    plot.legend.location = "top_left"
except Exception as e:
    print(e)  
show(plot)
colors = itertools.cycle(Category10[8])
plot = figure(width=1200, height=400, title="Carbon Monixide and Fast Ozone (if available, scaled / 1000)") 
plot.add_layout(Title(text="Time [Seconds since Midnight]", align="center"), "below")
plot.add_layout(Title(text="ppb", align="center"), "left")

try:
    plot.line(asc['Time'], asc['CO_PIC2401'], color=next(colors), legend_label='CO_PIC2401')
except Exception as e:
        print(e)
try:
    plot.line(asc['Time'], asc['CO_ARI']/1000, color=next(colors), legend_label='CO_ARI')
except Exception as e:
        print(e)
try:
    plot.legend.location = "top_left"
except Exception as e:
    print(e)  
show(plot)

## Cloud Probe Histograms

In [ ]:
cdp = []
for i in asc.columns:
    if i.startswith('CCDP'):
        cdp.append(i)
try:
    asc[cdp].describe()
except Exception as e:
    print(e)

In [ ]:
colors = itertools.cycle(Category10[8])
plot = figure(width=1200, height=400, title="CDP") 
plot.add_layout(Title(text="Time [Seconds since Midnight]", align="center"), "below")
plot.add_layout(Title(text="CDP", align="center"), "left")

for i in asc[cdp].columns:
    try:
        plot.line(asc['Time'], asc[cdp][i], color=next(colors), legend_label=i)
    except Exception as e:
        print(e)
try:
    plot.legend.location = "top_left"
except Exception as e:
    print(e)  
show(plot)